In [1]:
import win32com.client as win32
import pandas as pd
import os

In [2]:
#load the csv

df = pd.read_csv("C:/Users/Bavan Kumar/Downloads/New folder (16)/amazon.csv")

In [3]:
#changing the data format

df["actual_price"] = pd.to_numeric(df["actual_price"], errors="coerce")
df["discounted_price"] = pd.to_numeric(df["discounted_price"], errors="coerce")
df["discount_percentage"] = pd.to_numeric(df["discount_percentage"], errors="coerce")

In [4]:
# creating a new filepath to save the excel file

excel_path = os.path.abspath("C:/Users/Bavan Kumar/Desktop/pivot/bavan_pivot.xlsx")

In [5]:
if os.path.exists(excel_path):
    os.remove(excel_path)

In [6]:
#opening excel application using win32

df.to_excel(excel_path, index=False, sheet_name="Data")
excel = win32.gencache.EnsureDispatch('Excel.Application')
excel.Visible = True 

In [7]:
# if the workbook is open, it will close the application without saving

try:
    for wb in excel.Workbooks:
        if wb.FullName.lower() == excel_path.lower():
            wb.Close(SaveChanges=False)
except:
    pass

In [8]:
# selecting the pivot data range like excel

wb = excel.Workbooks.Open(excel_path)
ws = wb.Sheets("Data")

start_row = 1
start_col = 3
end_col = 6

last_row = ws.Cells(ws.Rows.Count, start_col).End(-4162).Row   # xlUp

last_col = ws.Cells(start_row, ws.Columns.Count).End(-4159).Column  # xlToLeft

data_range = ws.Range(ws.Cells(start_row, start_col), ws.Cells(last_row, end_col))


In [9]:
# selecting a new sheet and table destination for pivot table

pivot_sheet = wb.Sheets.Add()
pivot_sheet.Name = "PivotTable"

pc = wb.PivotCaches().Create(SourceType=1, SourceData=data_range)
pivot_table = pc.CreatePivotTable(
    TableDestination=pivot_sheet.Range("A3"),
    TableName="SalesPivot"
)

In [10]:
pt = pivot_sheet.PivotTables("SalesPivot")


In [11]:
#Adding row and data fields

pt.PivotFields("category").Orientation = 1 

pivot_table.AddDataField(pivot_table.PivotFields("actual_price"), "Sum of Actual Price", -4157) 
pivot_table.AddDataField(pivot_table.PivotFields("discounted_price"), "Sum of Discounted Price", -4157)
pivot_table.AddDataField(pivot_table.PivotFields("discount_percentage"), "Average Discount %", -4106) 

for df_item in pivot_table.DataFields:
    if "Price" in df_item.Name:
        df_item.NumberFormat = "₹#,##0.00"
    elif "Discount" in df_item.Name:
        df_item.NumberFormat = "0.00%"


#making it collapsible format if exists
pt.RowAxisLayout(1)  # 1 = xlCompactRow



In [12]:
#Preparing a name card for the total values


last_col = pivot_sheet.UsedRange.Columns.Count
label_col = last_col + 1  # Next to pivot (e.g., if pivot ends at E, this is F)

pivot_sheet.Cells(2, label_col).Value = "Grand Total - Actual Price"
pivot_sheet.Cells(2, label_col).Font.Bold = True
pivot_sheet.Cells(2, label_col).Interior.ColorIndex = 36  # Light yellow background
pivot_sheet.Cells(2, label_col).Borders.Weight = 2
pivot_sheet.Cells(2, label_col).HorizontalAlignment = -4108  # Center text

total_actual = df["actual_price"].sum()
pivot_sheet.Cells(2, label_col + 1).Value = total_actual
pivot_sheet.Cells(2, label_col + 1).NumberFormat = "₹#,##0.00"
pivot_sheet.Cells(2, label_col + 1).Font.Bold = True
pivot_sheet.Cells(2, label_col + 1).HorizontalAlignment = -4108  # Center

pivot_sheet.Cells(3, label_col).Value = "Grand Total - Discounted Price"
pivot_sheet.Cells(3, label_col).Font.Bold = True
pivot_sheet.Cells(3, label_col).Interior.ColorIndex = 35  # Light green background
pivot_sheet.Cells(3, label_col).Borders.Weight = 2
pivot_sheet.Cells(3, label_col).HorizontalAlignment = -4108  # Center text

# Ensure the column exists
if "discounted_price" in df.columns:
    total_discounted = df["discounted_price"].sum()
else:
    total_discounted = 0

pivot_sheet.Cells(3, label_col + 1).Value = total_discounted
pivot_sheet.Cells(3, label_col + 1).NumberFormat = "₹#,##0.00"
pivot_sheet.Cells(3, label_col + 1).Font.Bold = True
pivot_sheet.Cells(3, label_col + 1).HorizontalAlignment = -4108  # Center

#Add border for total values column
pivot_sheet.Range(pivot_sheet.Cells(2, label_col), pivot_sheet.Cells(3, label_col + 1)).Borders.Weight = 2


In [13]:
pivot_sheet.Columns.AutoFit()
wb.Save()
wb.Close(SaveChanges=True)
excel.Quit()

print(f" Collapsible Pivot Table created successfully at:\n{excel_path}")

 Collapsible Pivot Table created successfully at:
C:\Users\Bavan Kumar\Desktop\pivot\bavan_pivot.xlsx
